### Exercise 8.1 and 8.2

Program and compare the simple Eulerian advection schemes (upwind, central and downwind FD, Eqs. (8.4), (8.8), (8.9)) in 1D for the case illustrated in Fig. 8.6.

In fact, the three schemes yield almost identical results with numerical diffusion.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec

#### Step 1: Build the mesh and prescribe the initial wave form

In [ ]:
xsize = 150.0 # m
xnum = 151
xstp = xsize / (xnum-1)
xs = np.linspace(0.0, 150.0, xnum)

vx = 1 # 1m/s
ntimes = 800  # count of time steps
dx = 60.0  # total distance
t = dx / vx
dt = xstp / vx * dx / ntimes  # this corresponds to a small coefficient in the Corant condition

def get_initial_density(xs):
    '''
    get initial density on x coordinates
    Inputs:
        xs (list of double): x coordinates
    return:
        rho0s (list of doubles): initial densities
    '''
    xnum = xs.size
    rhos0 = np.zeros(xnum)
    # Defining initial density distribution 
    for i in range(xnum):
        # Background density
        rhos0[i] = 3000.0
        # Square wave
        if xs[i]>=3 and xs[i]<=23:
            rhos0[i]=3300
        # Triangular wave
        if xs[i] >= 43 and xs[i] <= 53:
            rhos0[i] = 3000+ (xs[i]-43) / 10 * 300.0
        if xs[i] >=53 and xs[i] <=63:
            rhos0[i] = 3300 - (xs[i]-53) / 10 * 300.0
    return rhos0


rhos0 = get_initial_density(xs)  # get initial density on nodes

#### Define the advection schemes
* upwinding
* downwinding
* central
* FCT
* marker in cell

Note that some boundary points need to be prescribed for each of these methods before applying finite difference.

The last 3 functions are for the "marker in cell" method.
They covers indexing of nearest node, advection and interpolating node properties, respectively.

In [ ]:
def exact_advection(rhos0, vx, t, xstp):
    '''
    The exact solution for advection
    Inputs:
        rhos0 (list of float): density for the last step
        vx: velocity
        dt: time step
        xstp: special resolution
    '''
    xnum = rhos0.size
    rhos = np.zeros(xnum)
    for i in range(xnum):
        x = i * xstp
        x0 = x - vx * t  # original place of lagrangian point
        if x0 < 0.0:
            rhos[i] = 3000.0  # material to the left all have backgroud density 
        else:
            n = int(x0 / xstp)
            rhos[i] = (x0 - xstp * n) / xstp * rhos0[n+1] + (xstp * (n+1) - x0) / xstp * rhos0[n]
    return rhos
    

def upwinding_advection(rhos0, vx, dt, xstp):
    '''
    upwinding advection scheme
    Inputs:
        rhos0 (list of float): density for the last step
        vx: velocity
        dt: time step
        xstp: special resolution
    '''
    xnum = rhos0.size
    rhos = np.zeros(xnum)
    rhos[0] = rhos0[0]  # value on the left boundary is maintained
    for i in range(1, xnum):
        rhos[i] = rhos0[i] - vx * dt * (rhos0[i] - rhos0[i-1]) / xstp
    return rhos


def central_advection(rhos0, vx, dt, xstp):
    '''
    central advection scheme
    Inputs:
        rhos0 (list of float): density for the last step
        vx: velocity
        dt: time step
        xstp: special resolution
    '''
    xnum = rhos0.size
    rhos = np.zeros(xnum)
    rhos[0] = rhos0[0]  # value on the left and right boundary is maintained
    rhos[xnum-1] = rhos0[xnum-1]
    for i in range(1, xnum-1):
        rhos[i] = rhos0[i] - vx * dt * (rhos0[i+1] - rhos0[i-1]) / xstp / 2.0
    return rhos


def downwinding_advection(rhos0, vx, dt, xstp):
    '''
    downwinding advection scheme
    Inputs:
        rhos0 (list of float): density for the last step
        vx: velocity
        dt: time step
        xstp: special resolution
    '''
    xnum = rhos0.size
    rhos = np.zeros(xnum)
    rhos[xnum-1] = rhos0[xnum-1]  # value on the right boundary is maintained
    for i in range(0, xnum-1):
        rhos[i] = rhos0[i] - vx * dt * (rhos0[i+1] - rhos0[i]) / xstp
    return rhos


def FCT_advection(rhos0, vx, dt, xstp):
    '''
    FCT advection scheme
    Inputs:
        rhos0 (list of float): density for the last step
        vx: velocity
        dt: time step
        xstp: special resolution
    '''
    D = xstp**2.0 / dt * (1.0/8 + 0.5 * (vx * dt / xstp)**2.0)
    xnum = rhos0.size
    rhos = np.zeros(xnum)
    rhos1 = np.zeros(xnum)
    rhos[0] = rhos0[0]    # values on the boundary, need 4 for the final solution
    rhos[1] = rhos0[1]
    rhos[xnum-2] = rhos0[xnum-2]
    rhos[xnum-1] = rhos0[xnum-1]
    rhos1[0] = rhos0[0]    # values on the boundary, need 2 for the transportation solution
    rhos1[xnum-1] = rhos0[xnum-1]
    for i in range(1, xnum-1):
        # values of rhos1, the transportation stage
        rhos1[i] = rhos0[i] - vx * dt * (rhos0[i+1] - rhos0[i-1]) / xstp / 2.0\
            + D * dt * (rhos0[i-1] - 2*rhos0[i] + rhos0[i+1]) / xstp**2.0
    for i in range(2, xnum-2):
        # values on half node, the anti-diffusion stage
        rho_minus_32 = rhos1[i-1] - rhos1[i-2]
        rho_minus_12 = rhos1[i] - rhos1[i-1]
        rho_plus_12 = rhos1[i+1] - rhos1[i]
        rho_plus_32 = rhos1[i+2] - rhos1[i+1]
        s_minus_12 = np.sign(rho_minus_12)
        s_plus_12 = np.sign(rho_plus_12)
        min_value_minus_12 = min(s_minus_12*rho_minus_32, 1.0/8.0*abs(rho_minus_12), s_minus_12*rho_plus_12)
        min_value_plus_12 = min(s_plus_12*rho_minus_12, 1.0/8.0*abs(rho_plus_12), s_plus_12*rho_plus_32)
        fad_minus_12 = s_minus_12 * max(0.0, min_value_minus_12)
        fad_plus_12 = s_plus_12 * max(0.0, min_value_plus_12)
        rhos[i] = rhos1[i] - fad_plus_12 + fad_minus_12
    return rhos


def nodeL_index(x, xstp):
    '''
    Get the nearest node to the left of a point x
    '''
    j = int(x / xstp)
    return j


def marker_in_cell_advection(xms0, vx, dt, xnum, xstp):
    '''
    marker in cell advection scheme
    Inputs:
        rhos0 (list of float): density for the last step
        vx: velocity
        dt: time step
        xnum: number of nodes
        xstp: special resolution
    '''
    xsize = (xnum - 1) * xstp  # size of the domain
    mnum = xms0.size
    xms = np.zeros(mnum)
    ids_on_nodes = [[] for i in range(xnum)]
    for i in range(mnum):
        xms[i] = xms0[i] + vx * dt
        if xms[i] < 0.0:
            # recycle left side
            xms[i] += xsize
        elif xms[i] > xsize:
            # recycle right side
            xms[i] -= xsize
        n = nodeL_index(xms[i], xstp) # add the index of this marker to nodes to the left and right
        ids_on_nodes[n].append(i)
        ids_on_nodes[n+1].append(i)
    return xms, ids_on_nodes


def marker_in_cell_property(xs, xms, rho_of_markers, ids_on_nodes):
    '''
    weight averaging of densities on markers to get the density field on nodes
    Inputs:
        xs (list of float): x coordinates
        xms (list of float): x coordinates of markers
        rho_of_markers: density of markers
        ids_on_nodes: ids of markers near a node
    '''
    xnum = xs.size
    rhos = np.zeros(xnum)
    for i in range(xnum):
        sum_dist = 0.0
        for id in ids_on_nodes[i]:
            dist = abs(xms[id] - xs[i])
            rhos[i] += rho_of_markers[id] * dist
            sum_dist += dist
        rhos[i] /= sum_dist 
    return rhos



#### Step 3: advection

Here, an array is initiated for each method.
Advection is handled by looping the time steps.

In [ ]:
rhos_up = rhos0.copy()
rhos_central = rhos0.copy()
rhos_down = rhos0.copy()
rhos_FCT = rhos0.copy()
rhos_mic = rhos0.copy()
rhos = rhos0.copy()
rhos = exact_advection(rhos, vx, t, xstp)
n_marker = 200  # number of markers
xms = np.linspace(0, xsize, n_marker)  # initial positions of markers
rho_of_markers = get_initial_density(xms)  # initial density of markers

# advection by time step
for i in range(ntimes):
    rhos_up = upwinding_advection(rhos_up, vx, dt, xstp)
    rhos_central = central_advection(rhos_up, vx, dt, xstp)
    rhos_down = downwinding_advection(rhos_up, vx, dt, xstp)
    rhos_FCT = FCT_advection(rhos_FCT, vx, dt, xstp)
    xms, ids_on_nodes = marker_in_cell_advection(xms, vx, dt, xnum, xstp)
    rhos_mic = marker_in_cell_property(xs, xms, rho_of_markers, ids_on_nodes)



#### Plot

In [ ]:
fig = plt.figure(tight_layout=True, figsize=(5, 20))
gs = gridspec.GridSpec(4, 1)
ax = fig.add_subplot(gs[0, 0])  # initial
ax.plot(xs, rhos0, 'b')
ax.set_xlabel("x [m]")
ax.set_ylabel("Density [kg/m3]")
ax.set_xlim(0.0, 80.0)
ax.set_title("t = 0.0 s")
ax = fig.add_subplot(gs[1, 0])  # winding schemes 
ax.plot(xs, rhos, 'b')
ax.plot(xs, rhos_up, 'r.', label="upwinding") # upwinding
ax.plot(xs, rhos_central, 'b.', label="central") # central
ax.plot(xs, rhos_down, 'g.', label="downwinding")  # downwinding
ax.set_xlabel("x [m]")
ax.set_ylabel("Density [kg/m3]")
ax.set_xlim(dx, 80.0 + dx)
ax.set_title("Winding, t = %.1f s" % t)
ax.legend()
ax = fig.add_subplot(gs[2, 0])  # FCT
ax.plot(xs, rhos, 'b')
ax.plot(xs, rhos_FCT, 'r.', label="FCT")
ax.set_xlabel("x [m]")
ax.set_ylabel("Density [kg/m3]")
ax.set_xlim(dx, 80.0 + dx)
ax.set_title("FCT, t = %.1f s" % t)
ax.legend()
ax = fig.add_subplot(gs[3, 0])  # method in cell
ax.plot(xs, rhos, 'b')
ax.plot(xs, rhos_mic, 'r.', label="marker in cell")
ax.set_xlabel("x [m]")
ax.set_ylabel("Density [kg/m3]")
ax.set_xlim(dx, 80.0 + dx)
ax.set_title("MiC, t = %.1f s" % t)
ax.legend()
fig.show()